# technical scanner

In [ ]:
# Technicals Scanner #
fmr_chat_pump = []
for sym in dataset.Symbol:
  try:
    data = datacapture("1d",sym,today-BDay(252),today)
    data['prev_close'] = data.close.shift(periods = 1)

    # 60 Day Ave Vol > $1M #
    ave_vol_60d = 1 if (data.volume[-60:].mean() * data.close[-60:].mean()) >= 1000000 else 0

    # 5 Day Ave Vol > $100K #
    ave_vol_5d = 1 if (data.volume[-5:].mean() * data.close[-5:].mean()) >= 100000 else 0

    # Former MDR #
    data['mdr'] = (data.high.shift(periods = 3) / data.open) - 1
    frm_run = 1 if len(data[data.mdr >= 2]) > 0 else 0

    # Short-Term Uptrend #
    trend = 1 if (data.close.iloc[-1] / data.close.iloc[-6]) - 1 >= 0.25 else 0
    if trend == 1:
      five_day_high = data.high[-5:].max()
      dist_from_high = (data.close.iloc[-1] / five_day_high) - 1
      near_high = 1 if dist_from_high >= -0.2 else 0
    else:
      near_high = 0

    if pm_gapup == 1 and ave_vol_60d == 1 and ave_vol_5d == 1 and frm_run == 1 and near_high == 1:
      fmr_chat_pump.append(sym)
    else:
      pass
  except:
    pass
len(fmr_chat_pump)

In [ ]:
# Secondary Run #
sec_run = []
for sym in news_pump:
  data_int = datacapture("1d",sym,today-BDay(3),today)
  data_int['date'] = data_int.timestamp.apply(lambda a: datetime.datetime.fromtimestamp(a))

  # Big O/N gap up day prior + abnormal vol #
  o_n_gap = 1 if (data_int.high[1] / data_int.close[0]) - 1 >= 0.5 else 0

  # Consolidation near gap up high #
  cons_near_high = (data_int.close.iloc[-1] / data_int.high[1]) - 1 >= -0.10

  # Volume Accumulation on Consolidation #
  vol_on_gap_day = data_int.volume[1]
  vol_on_cons_day = data_int.volume.iloc[-1]
  vol_acc = 1 if vol_on_gap_day - vol_on_cons_day > 0 else 0

In [ ]:
look for a swing setup that shows
1) long term downtrend + short term consoliation near the lows + short term uptrend with higher lows to confirm trend reversal
2) during the short term uptrend, price must break above short-term resistance created from short term consolidation to prove bulls in control
